In [ ]:
# ! pip install jupyter openpyxl seaborn matplotlib
# ! pip install -U sentence-transformers
# ! pip3 install alibi scikit-learn-extra
! pip3 install wordcloud

In [ ]:
import datetime
import sys
from pathlib import Path

import numpy as np

print(f"UTC now= '{datetime.datetime.utcnow().isoformat().split('.')[0]}'")

print(f'Virtualenv used: {sys.executable}')

import pandas as pd
import ast
import pickle

from sklearn.cluster import KMeans

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

from sklearn import tree
import nltk
nltk.download('punkt')

In [ ]:
def _save_pickle(df, outfile, results_folder):
  if results_folder:
    results_folder_local = f"{str(Path('~').expanduser().resolve())}/{results_folder}"
    # results_folder_local = results_folder
  else:
    results_folder_local = f"{str(Path('~').expanduser().resolve())}/data"
  Path(results_folder_local).mkdir(parents=True, exist_ok=True)
  fname = f"{results_folder_local}/{outfile}.pickle"
  print(fname)
  with open(fname, 'wb') as f:
    pickle.dump(df, f)


def get_LSA(df, column_to_ommit, how_many_dimmension):
  svd = TruncatedSVD(how_many_dimmension) # >~ 70%
  normalizer = Normalizer(copy=False)
  lsa = make_pipeline(svd, normalizer)

  df_edges_lsa = pd.DataFrame(lsa.fit_transform(df.loc[:, df.columns != column_to_ommit]))
  print(f"explained variance perc = {svd.explained_variance_ratio_.sum()*100}%")

  df_edges_lsa = pd.concat([df['article_which_cities'], df_edges_lsa], axis=1, join="inner")
  df_edges_lsa['cluster_kmeans'] = -1
  return df_edges_lsa


def k_means(df, column_to_ommit, how_many_clusters):
  kmeans = KMeans(n_clusters=how_many_clusters, random_state=2022).fit(df.loc[:, (df.columns != column_to_ommit) & (df.columns != 'cluster_kmeans')])
  # kmeans.labels_
  # print(pd.DataFrame(kmeans.cluster_centers_).reset_index())
  # print(pd.DataFrame(kmeans.cluster_centers_).reset_index()['index'])
  df['cluster_kmeans'] = kmeans.labels_
  # df_only_one_hots = df.loc[:, (df.columns != 'article_which_cities') & (df.columns != 'cluster_kmeans')]
  # df[['article_which_cities', 'cluster_kmeans']]
  return df


def get_TSNE_and_PCA_embeddings(df_edges, column_to_ommit):
  df_edges_independent_vars = df_edges.loc[:, (df_edges.columns != column_to_ommit) & (df_edges.columns != 'cluster_kmeans')]

  df_tsne = pd.DataFrame(TSNE(n_components=2).fit_transform(df_edges_independent_vars))
  df_tsne['cluster'] = df_edges['cluster_kmeans']
  df_tsne.columns = ['x1', 'x2', 'cluster']
  df_tsne['article_which_cities'] = df_edges['article_which_cities']

  df_pca = pd.DataFrame(PCA(n_components=2).fit_transform(df_edges_independent_vars))
  df_pca['cluster'] = df_edges['cluster_kmeans']
  df_pca.columns = ['x1', 'x2', 'cluster']
  df_pca['article_which_cities'] = df_edges['article_which_cities']

  scaler = MinMaxScaler()
  df_tsne[['x1', 'x2']] = scaler.fit_transform(df_tsne[['x1', 'x2']])
  df_pca[['x1', 'x2']] = scaler.fit_transform(df_pca[['x1', 'x2']])

  return df_tsne, df_pca


def wrap_by_word(s, n):
  '''returns a string where \\n is inserted between every n words'''
  a = s.split()
  ret = ''
  for i in range(0, len(a), n):
    ret += ' '.join(a[i:i+n]) + '\n'

  return ret

# x = wrap_by_word('There is a dog and fox fighting in the park and there is an apple falling down.', 4)
# print(x)
def get_article_info(df_with_article_which_cities_column, column):
  df_nodes_cp = df_nodes.copy()
  df_nodes_cp['article_which_cities'] = df_nodes_cp['ID'].apply(lambda x: f"{x:02d}")
  df_merged = df_with_article_which_cities_column.merge(df_nodes_cp, on='article_which_cities', how='left')

  return df_merged['article_which_cities'] + " (" + df_merged['Year'].apply(lambda x: str(x)) + ") " + ": \n" + df_merged[column].apply(lambda x: wrap_by_word(x, 3))

# get_article_info(df_tsne_lsa, column='Title')

def label_point(x, y, val, ax, factor):
  a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
  for i, point in a.iterrows():
    ax.text(point['x'] + 1*factor, point['y'] - 10*factor, str(point['val']))


def visualise_clusters(df_2d_embedding, factor=1/200):
  _fig, _ax = plt.subplots(1, 1, figsize=(12*2,6*2))
  sns.scatterplot(data=df_2d_embedding, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, s=120, palette="deep", ax=_ax)
  label_point(df_2d_embedding['x1'], df_2d_embedding['x2'], get_article_info(df_2d_embedding, column='Title'), _ax, factor)
  # display(_fig)


In [ ]:
# df_nodes = pd.read_excel('/media/mmozolewski/m.mozolewski@gma/Documents/Doktorat/Parisa/xai-survey/RC6 nodes info(Abstract).xlsx', sheet_name=0)
# df_edges = pd.read_excel('/media/mmozolewski/m.mozolewski@gma/Documents/Doktorat/Parisa/xai-survey/Data.xlsx', sheet_name=0)
df_nodes = pd.read_excel('notebooks/xai-survey/RC6 nodes info(Abstract).xlsx', sheet_name=0)
df_edges = pd.read_excel('notebooks/xai-survey/Data.xlsx', sheet_name=0)

In [ ]:
df_nodes['Title_and_Abstract'] = df_nodes['Title'] + ' ' + df_nodes['Abstract']
df_nodes[['ID', 'Title', 'Abstract', 'CitationCount', 'RepetitionCount']]

In [ ]:
df_nodes.loc[df_nodes['Abstract'].isna(), ]

In [ ]:
df_edges.loc[(df_edges['node 1']==36) | (df_edges['node 2']==36), ]

In [ ]:
# print(df_edges['link type'].dtype)
# df_edges.loc[32, 'link type'][0]
# ast.literal_eval(df_edges.loc[32, 'link type'])
df_edges['link type'] = df_edges['link type'].apply(lambda x: "['unknown']" if x == "[]" else x)
df_edges['link type'] = df_edges['link type'].apply(lambda x: ast.literal_eval(x))
df_edges

In [ ]:
# Plan :
# 4 relacje
# 50 artów
#
# id | is_methodology_01 | is_result_01 | is_result_01 |
# 00       true                false

# Opcja: SVD (LSA) -> jak PCA, TF-IDF
# Normalizacja
# k-means
# KnAC (objaśnienia na abstraktach)

In [ ]:
df_edges_flatten = []
for index, row in df_edges.iterrows():
  for link_type in row['link type']:
    df_edges_flatten.append((f"{row['node 1']:02d}", f"{row['node 2']:02d}_{link_type}", row['node 1'], row['node 2'], link_type))
df_edges_flatten = pd.DataFrame(df_edges_flatten)
df_edges_flatten.sort_values(by=[2, 3, 4], inplace=True)
df_edges_flatten

In [ ]:
# pd.pivot_table(df_edges_flatten, index=0, values=2, columns=1, aggfunc='-'.join)
df_edges_flatten_one_hot = pd.get_dummies(df_edges_flatten[1])
df_edges_flatten_one_hot = pd.concat([df_edges_flatten, df_edges_flatten_one_hot], axis=1, join="inner")
df_edges_flatten_one_hot

## One-hot features

In [ ]:
df_edges_one_hot = df_edges_flatten_one_hot.groupby(0).agg(lambda x: x.sum()).reset_index()
df_edges_one_hot.rename(columns={0: "article_which_cities"}, inplace=True)
del df_edges_one_hot[1]
del df_edges_one_hot[2]
del df_edges_one_hot[3]
del df_edges_one_hot[4]
df_edges_one_hot['cluster_kmeans'] = -1
df_edges_one_hot

## LSA features on One-HOT

In [ ]:
df_edges_lsa = get_LSA(df_edges_one_hot, 'article_which_cities', 12)
df_edges_lsa

## Sentence-BERT embeddings

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
df_edges_bert = pd.DataFrame(model.encode(df_nodes['Abstract'].tolist()))
df_edges_bert = pd.concat([df_nodes[['ID']], df_edges_bert], axis=1, join="inner").rename(columns={'ID':'article_which_cities'})
df_edges_bert['article_which_cities'] = df_edges_bert['article_which_cities'].apply(lambda x: f"{x:02d}")
df_edges_bert.head(3)

In [ ]:
df_edges_bert = pd.merge(df_edges_one_hot['article_which_cities'], df_edges_bert, left_on='article_which_cities', right_on='article_which_cities')
df_edges_bert['cluster_kmeans'] = -1
df_edges_bert

# LSA features on Sentence-BERT embeddings

In [ ]:
df_edges_bert_lsa = get_LSA(df_edges_bert, 'article_which_cities', 13)
df_edges_bert_lsa

# SpaCy embeddings

In [ ]:
import spacy
# from alibi.utils.download import spacy_model

model_spacy = 'en_core_web_lg'
# spacy_model(model=model_spacy)
spacy_nlp = spacy.load(model_spacy)

In [ ]:
df_edges_spacy = pd.DataFrame([doc.vector for doc in spacy_nlp.pipe([str(x)  for x in df_nodes['Title_and_Abstract'].tolist()])])
df_edges_spacy = pd.concat([df_nodes[['ID']], df_edges_spacy], axis=1, join="inner").rename(columns={'ID':'article_which_cities'})
df_edges_spacy['article_which_cities'] = df_edges_spacy['article_which_cities'].apply(lambda x: f"{x:02d}")
df_edges_spacy.head(3)

In [ ]:
df_edges_spacy = pd.merge(df_edges_one_hot['article_which_cities'], df_edges_spacy, left_on='article_which_cities', right_on='article_which_cities')
df_edges_spacy['cluster_kmeans'] = -1
df_edges_spacy

# LSA for SpaCy

In [ ]:
df_edges_spacy_lsa = get_LSA(df_edges_spacy, 'article_which_cities', 11)
df_edges_spacy_lsa

# K-means clustering

In [ ]:
df_edges_one_hot = k_means(df_edges_one_hot, 'article_which_cities', 3)
df_edges_lsa = k_means(df_edges_lsa, 'article_which_cities', 3)
df_edges_bert = k_means(df_edges_bert, 'article_which_cities', 3)
df_edges_bert_lsa = k_means(df_edges_bert_lsa, 'article_which_cities', 3)
df_edges_spacy_lsa = k_means(df_edges_spacy_lsa, 'article_which_cities', 3)

In [ ]:
df_tsne_one_hots, df_pca_one_hots = get_TSNE_and_PCA_embeddings(df_edges_one_hot, 'article_which_cities')
df_tsne_lsa, df_pca_lsa = get_TSNE_and_PCA_embeddings(df_edges_lsa, 'article_which_cities')
df_tsne_bert, df_pca_bert = get_TSNE_and_PCA_embeddings(df_edges_bert, 'article_which_cities')
df_tsne_bert_lsa, df_pca_bert_lsa = get_TSNE_and_PCA_embeddings(df_edges_bert_lsa, 'article_which_cities')
df_tsne_spacy_lsa, df_pca_spacy_lsa = get_TSNE_and_PCA_embeddings(df_edges_spacy_lsa, 'article_which_cities')

In [ ]:
# _save_pandas_pickle(df_tsne_one_hots, 'df_tsne_one_hots', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_pca_one_hots, 'df_pca_one_hots', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_tsne_lsa, 'df_tsne_lsa', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_pca_lsa, 'df_pca_lsa', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_tsne_bert, 'df_tsne_bert', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_pca_bert, 'df_pca_bert', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_tsne_bert_lsa, 'df_tsne_bert_lsa', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_pca_bert_lsa, 'df_pca_bert_lsa', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_tsne_spacy_lsa, 'df_pca_bert_lsa', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_pca_spacy_lsa, 'df_pca_bert_lsa', "notebooks/xai-survey/clusters-embeddings")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12*2,6*2))
sns.scatterplot(data=df_tsne_one_hots, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[0,0])
ax[0,0].set_title('One-Hot - Visualized on TSNE 2D')
sns.scatterplot(data=df_tsne_lsa, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[0,1])
ax[0,1].set_title('LSA - Visualized on TSNE 2D')
# sns.scatterplot(data=df_tsne_bert, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[0,2])
# ax[0,2].set_title('** BERT - Visualized on TSNE 2D')

sns.scatterplot(data=df_pca_one_hots, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[1,0])
ax[1,0].set_title('One-Hot - Visualized on PCA 2D')
sns.scatterplot(data=df_pca_lsa, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[1,1])
ax[1,1].set_title('LSA - Visualized on PCA 2D')
# sns.scatterplot(data=df_pca_bert, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, palette="deep", s=100, ax=ax[1,2])
# ax[1,2].set_title('** BERT - Visualized on PCA 2D')

fig.suptitle('Clustering result: citations data (One-Hot, LSA) \n (visualized using TSNE2D vs. PCA2D)') #and abstracts (BERT)
# display(fig)

In [ ]:
### some articles, like 20 does not cite any other article, so they are not on this graph ###
visualise_clusters(df_tsne_lsa)
# visualise_clusters(df_pca_lsa)

In [ ]:
## clustering on BERT emmbedings - not good:
# visualise_clusters(df_tsne_bert)
visualise_clusters(df_tsne_bert_lsa)
# visualise_clusters(df_pca_bert)
# visualise_clusters(df_pca_bert_lsa)

In [ ]:
visualise_clusters(df_tsne_spacy_lsa)

# Clustering on citation + abstract
## clustering on both citation data and BERT

In [ ]:
df_edges_bert_lsa_and_lsa = pd.concat([df_edges_bert_lsa, df_edges_lsa.loc[:, (df_edges_lsa.columns != 'article_which_cities') & (df_edges_lsa.columns != 'cluster_kmeans')]], axis=1)
df_edges_bert_lsa_and_lsa = k_means(df_edges_bert_lsa_and_lsa, 'article_which_cities', 7)
df_edges_bert_lsa_and_lsa_tsne, df_edges_bert_lsa_and_lsa_pca = get_TSNE_and_PCA_embeddings(df_edges_bert_lsa_and_lsa, 'article_which_cities')
# _save_pandas_pickle(df_edges_bert_lsa_and_lsa_tsne, 'df_edges_bert_lsa_and_lsa_tsne', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_edges_bert_lsa_and_lsa_pca, 'df_edges_bert_lsa_and_lsa_pca', "notebooks/xai-survey/clusters-embeddings")

visualise_clusters(df_edges_bert_lsa_and_lsa_tsne)
# df_edges_bert_lsa_and_lsa_tsne

## clustering on both citation data and spacy

In [ ]:
df_edges_spacy_lsa_and_lsa = pd.concat([df_edges_spacy_lsa, df_edges_lsa.loc[:, (df_edges_lsa.columns != 'article_which_cities') & (df_edges_lsa.columns != 'cluster_kmeans')]], axis=1)
df_edges_spacy_lsa_and_lsa = k_means(df_edges_spacy_lsa_and_lsa, 'article_which_cities', 7)
df_edges_spacy_lsa_and_lsa_tsne, df_edges_spacy_lsa_and_lsa_pca = get_TSNE_and_PCA_embeddings(df_edges_spacy_lsa_and_lsa, 'article_which_cities')
# _save_pandas_pickle(df_edges_spacy_lsa_and_lsa_tsne, 'df_edges_spacy_lsa_and_lsa_tsne', "notebooks/xai-survey/clusters-embeddings")
# _save_pandas_pickle(df_edges_spacy_lsa_and_lsa_pca, 'df_edges_spacy_lsa_and_lsa_pca', "notebooks/xai-survey/clusters-embeddings")

visualise_clusters(df_edges_spacy_lsa_and_lsa_tsne)

In [ ]:
# TODO :
# Zamiast BERT - TF-IDF, W2V
# clamp -> objaśnie klustw
# expert clusters from Parisa  -> knAC
# inxai -> dodac NLU, metryki

In [ ]:
df_nodes

## explain clusters on citations with abstracts

In [ ]:
def add_cluster_col(df_nodes, df_with_cluster_kmeans_column):
  df_nodes['ID-str'] = df_nodes['ID'].apply(lambda x: f"{x:02d}")
  df_edges_for_xai = pd.merge(df_with_cluster_kmeans_column.loc[:, ['article_which_cities', 'cluster_kmeans']], df_nodes[['ID-str', 'Title', 'Abstract']], left_on='article_which_cities', right_on='ID-str')[['article_which_cities', 'cluster_kmeans', 'Title', 'Abstract']]
  df_edges_for_xai['Title_and_Abstract'] = df_edges_for_xai['Title'] + ' ' + df_edges_for_xai['Abstract']
  return df_edges_for_xai

df_edges_for_xai = add_cluster_col(df_nodes, df_edges_lsa)
df_edges_for_xai

In [ ]:
all_stopwords = spacy_nlp.Defaults.stop_words
all_stopwords.update(['et', 'al', 'approach', 'model', 'method', 'explanation']) ## TODO get better list from bigger corpus of abstracts
'method' in all_stopwords

In [ ]:
###  TODO
# mniej klastrow
# decision tree classfier
# potem odwrocic TF-IDF

In [ ]:
## TODO - use Cross Validation
train, test, train_labels, test_labels = train_test_split(
  df_edges_for_xai['Abstract'], df_edges_for_xai['cluster_kmeans'], test_size=.3) # , random_state=202205
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer(max_features=20, stop_words=all_stopwords, min_df=2, max_df=.2, ngram_range=(1,2)) # CountVectorizer(max_features=100, stop_words=all_stopwords) #
vectorizer.fit(train)

clf = DecisionTreeClassifier() # xgb.XGBClassifier(use_label_encoder=True) # LogisticRegression(solver='liblinear') #
clf.fit(vectorizer.transform(train), train_labels)
predict_fn = lambda x: clf.predict(vectorizer.transform(x))

preds_train = predict_fn(train)
preds_test = predict_fn(test)
print('Train accuracy', accuracy_score(train_labels, preds_train))
print('Test accuracy', accuracy_score(test_labels, preds_test))

In [ ]:
print(preds_train)
print(preds_test)
# vectorizer.transform(["The problem of attributing the prediction of a deep network to its input features"])
# predict_fn(["The problem of attributing the prediction of a deep network to its input features"])

In [ ]:
response = vectorizer.fit_transform(df_edges_for_xai['Abstract'])
df_edges_for_xai_vectors = pd.DataFrame(response.toarray(), columns=vectorizer.get_feature_names())
print(vectorizer.get_feature_names())

In [ ]:
# _save_pickle(clf, 'clf_DecisionTreeClassifier', "notebooks/xai-survey/clusters-embeddings")

In [ ]:
# visualization
fig = plt.figure(figsize=(16, 8))
vis = tree.plot_tree(clf, feature_names = vectorizer.get_feature_names(), class_names = ["0", "1", "2"], max_depth=5, fontsize=9, proportion=True, filled=True, rounded=True)

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.tokenize import word_tokenize

for cluster in [0, 1, 2]:
  print (cluster)
  # Start with one review:
  text = word_tokenize(" ".join(df_edges_for_xai.loc[df_edges_for_xai['cluster_kmeans']==cluster, 'Abstract'].tolist()))
  text = " ".join([w for w in text if not w.lower() in all_stopwords])

  # Create and generate a word cloud image:
  wordcloud = WordCloud().generate(text)

  # Display the generated image:
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()

In [ ]:
## TODO
# - Use expert knowledge??
# - Use citations : cluster 1 == cities_art_12
#   - common words in abstract for cluster (~ TFID)

# Wizualizacja - czym różnią się clustry:
# - word cloud dla clustra + polaczenia między clustrami - czym się różnią

# Counterfactual